<div >
<img src = "../banner.jpg" />
</div>

# Desbalance de Clases

En problemas de clasificación, las frecuencias relativas de las clases pueden tener un impacto significativo en la efectividad del modelo. Se produce un desequilibrio o desbalance cuando una o más clases tienen proporciones muy bajas en los datos de entrenamiento en comparación con las otras clases. 

El desbalance puede estar presente en cualquier conjunto de datos o aplicación y, por lo tanto, el profesional debe ser consciente de las implicaciones de modelar este tipo de datos.

Según qué tan desbalanceada sea su base se puede clasificar el problema en diferentes categorías. Una buena regla del pulgar se encuentra en la siguiente tabla:

| **Grado de desbalance** | **Proporción de la clase minoritaria** |
|:----------------------:|:--------------------------------------:|
|          Leve          |       20-40% de las observaciones      |
|        Moderado        |       1-20% de las observaciones       |
|         Extremo        |        <1% de las observaciones        |

## Preparar la base

In [1]:
#Cargar librerías 
require("pacman")
p_load("tidyverse")


Loading required package: pacman



In [2]:
#Leer los datos 
credit <- readRDS(url("https://github.com/ignaciomsarmiento/datasets/blob/main/credit_class.rds?raw=true"))
#mutacion de factores
credit<-credit %>% mutate(Default=factor(Default,levels=c(0,1),labels=c("No","Si")),
                          history=factor(history,levels=c("good","poor","terrible"),labels=c("buena","mala","terrible")),
                          foreign=factor(foreign,levels=c("foreign","german"),labels=c("extranjero","aleman")),
                          purpose=factor(purpose,levels=c("newcar","usedcar","goods/repair","edu", "biz" ),labels=c("auto_nuevo","auto_usado","bienes","educacion","negocios")))         

head(credit)

,Default,duration,amount,installment,age,history,purpose,foreign,rent
,<fct>,<int>,<int>,<int>,<int>,<fct>,<fct>,<fct>,<fct>
1,No,6,1169,4,67,terrible,bienes,extranjero,FALSE
2,Si,48,5951,2,22,mala,bienes,extranjero,FALSE
3,No,12,2096,2,49,terrible,educacion,extranjero,FALSE
4,No,42,7882,2,45,mala,bienes,extranjero,FALSE
5,Si,24,4870,3,53,mala,auto_nuevo,extranjero,FALSE
6,No,36,9055,2,35,mala,educacion,extranjero,FALSE


In [3]:
p_load("caret")
dmy <- dummyVars(" ~ .", data = credit)
head(dmy)

$call
dummyVars.default(formula = " ~ .", data = credit)

$form
~.
<environment: 0x7fe67491a8c8>

$vars
[1] "Default"     "duration"    "amount"      "installment" "age"        
[6] "history"     "purpose"     "foreign"     "rent"       

$facVars
[1] "Default" "history" "purpose" "foreign" "rent"   

$lvls
$lvls$Default
[1] "No" "Si"

$lvls$history
[1] "buena"    "mala"     "terrible"

$lvls$purpose
[1] "auto_nuevo" "auto_usado" "bienes"     "educacion"  "negocios"  

$lvls$foreign
[1] "extranjero" "aleman"    

$lvls$rent
[1] NA      "FALSE" "TRUE" 


$sep
[1] "."


In [4]:
credit <- data.frame(predict(dmy, newdata = credit))
credit<- credit  %>% mutate(Default=factor(Default.Si,levels=c(0,1),labels=c("No","Si")))
head(credit)

,Default.No,Default.Si,duration,amount,installment,age,history.buena,history.mala,history.terrible,purpose.auto_nuevo,purpose.auto_usado,purpose.bienes,purpose.educacion,purpose.negocios,foreign.extranjero,foreign.aleman,rent.NA,rent.FALSE,rent.TRUE,Default
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
1,1,0,6,1169,4,67,0,0,1,0,0,1,0,0,1,0,0,1,0,No
2,0,1,48,5951,2,22,0,1,0,0,0,1,0,0,1,0,0,1,0,Si
3,1,0,12,2096,2,49,0,0,1,0,0,0,1,0,1,0,0,1,0,No
4,1,0,42,7882,2,45,0,1,0,0,0,1,0,0,1,0,0,1,0,No
5,0,1,24,4870,3,53,0,1,0,1,0,0,0,0,1,0,0,1,0,Si
6,1,0,36,9055,2,35,0,1,0,0,0,0,1,0,1,0,0,1,0,No


In [5]:
prop.table(table(credit$Default))


 No  Si 
0.7 0.3 

### División de la muestra

- El objetivo es predecir bien fuera de muestra

- No queremos sobreajustar a la muestra
  
- Vamos a definir 3 bases

  - Muestra de entrenamiento: vamos a estimar los modelos, buscar parámetros, etc.
  
  -  Muestra de evaluación (pequeña): para desarrollar técnicas de post procesamiento
  
  -  Muestra de prueba que solo vamos a usar para evaluar los modelos


In [6]:
## First, split the training set 
set.seed(156)
split1 <- createDataPartition(credit$Default, p = .7)[[1]]
length(split1)
head(split1, n=20)

[1] 700

[1]  1  2  3  4  5  8 11 12 13 14 15 17 18 19 20 21 22 23 25 26

In [7]:
other     <- credit[-split1,]
training  <- credit[ split1,]

In [8]:
# Evaluación y prueba (testing)
set.seed(934)
split2 <- createDataPartition(other$Default, p = 1/3)[[1]]
evaluation  <- other[ split2,]
testing     <- other[-split2,]

In [9]:
dim(training)
dim(testing)
dim(evaluation)

[1] 700  20

[1] 200  20

[1] 100  20

In [10]:
#Logit

In [11]:
fiveStats <- function(...) c(twoClassSummary(...), defaultSummary(...))
ctrl<- trainControl(method = "cv",
                     number = 5,
                     summaryFunction = fiveStats,
                     classProbs = TRUE,
                     verbose=FALSE,
                     savePredictions = T)
#logit
set.seed(1410)
mylogit_caret <- train(Default~duration+amount+installment+age+
                       history.buena+history.mala+
                       purpose.auto_nuevo+purpose.auto_usado+purpose.bienes+purpose.educacion+
                       foreign.extranjero+
                       +rent.TRUE, 
                       data = training, 
                       method = "glm",
                       trControl = ctrl,
                       family = "binomial", 
                       metric = 'ROC')




mylogit_caret

Generalized Linear Model 

700 samples
 12 predictor
  2 classes: 'No', 'Si' 

No pre-processing
Resampling: Cross-Validated (5 fold) 
Summary of sample sizes: 560, 560, 560, 560, 560 
Resampling results:

  ROC        Sens       Spec       Accuracy   Kappa    
  0.7308066  0.9102041  0.3047619  0.7285714  0.2456281


## Model Tuning: Maximizar la capacidad predictiva del modelo


El enfoque más simple para contrarrestar los efectos negativos del desequilibrio de clases es ajustar el modelo para maximizar la precisión de las clases minoritarias.

Para nuestro ejemplo, ajustar el modelo para maximizar la sensibilidad puede ayudar a desensibilizar el proceso de entrenamiento al alto porcentaje de datos sin default en el conjunto de entrenamiento.

In [12]:
#logit
set.seed(1410)
mylogit_caret <- train(Default~duration+amount+installment+age+
                       history.buena+history.mala+
                       purpose.auto_nuevo+purpose.auto_usado+purpose.bienes+purpose.educacion+
                       foreign.extranjero+
                       +rent.TRUE, 
                       data = training, 
                       method = "glm",
                       trControl = ctrl,
                       family = "binomial", 
                       metric = 'Spec')

mylogit_caret

Generalized Linear Model 

700 samples
 12 predictor
  2 classes: 'No', 'Si' 

No pre-processing
Resampling: Cross-Validated (5 fold) 
Summary of sample sizes: 560, 560, 560, 560, 560 
Resampling results:

  ROC        Sens       Spec       Accuracy   Kappa    
  0.7308066  0.9102041  0.3047619  0.7285714  0.2456281


In [13]:
#Lasso
lambda_grid <- 10^seq(-4, 0.01, length = 10) #en la practica se suele usar una grilla de 200 o 300


set.seed(1410)
mylogit_lasso_spec <- train(Default~duration+amount+installment+age+
                       history.buena+history.mala+
                       purpose.auto_nuevo+purpose.auto_usado+purpose.bienes+purpose.educacion+
                       foreign.extranjero+
                       +rent.TRUE,
  data = training, 
  method = "glmnet",
  trControl = ctrl,
  family = "binomial", 
  metric = "Spec",
  tuneGrid = expand.grid(alpha = 0,lambda=lambda_grid), 
  preProcess = c("center", "scale")
)
mylogit_lasso_spec

glmnet 

700 samples
 12 predictor
  2 classes: 'No', 'Si' 

Pre-processing: centered (12), scaled (12) 
Resampling: Cross-Validated (5 fold) 
Summary of sample sizes: 560, 560, 560, 560, 560 
Resampling results across tuning parameters:

  lambda        ROC        Sens       Spec        Accuracy   Kappa     
  0.0001000000  0.7318756  0.9142857  0.28571429  0.7257143  0.23176719
  0.0002789687  0.7318756  0.9142857  0.28571429  0.7257143  0.23176719
  0.0007782356  0.7318756  0.9142857  0.28571429  0.7257143  0.23176719
  0.0021710342  0.7318756  0.9142857  0.28571429  0.7257143  0.23176719
  0.0060565070  0.7318756  0.9142857  0.28571429  0.7257143  0.23176719
  0.0168957618  0.7327017  0.9142857  0.27619048  0.7228571  0.22125732
  0.0471338954  0.7335763  0.9326531  0.26190476  0.7314286  0.23129637
  0.1314888384  0.7347425  0.9571429  0.14285714  0.7128571  0.12505390
  0.3668127682  0.7336249  0.9918367  0.02380952  0.7014286  0.02124838
  1.0232929923  0.7298834  1.0000000  0.0

### Accuracy

In [14]:
#Lasso
lambda_grid <- 10^seq(-4, 0.01, length = 10) #en la practica se suele usar una grilla de 200 o 300


set.seed(1410)
mylogit_lasso_acc <- train(Default~duration+amount+installment+age+
                       history.buena+history.mala+
                       purpose.auto_nuevo+purpose.auto_usado+purpose.bienes+purpose.educacion+
                       foreign.extranjero+
                       +rent.TRUE,
  data = training, 
  method = "glmnet",
  trControl = ctrl,
  family = "binomial", 
  metric = "Accuracy",
  tuneGrid = expand.grid(alpha = 0,lambda=lambda_grid), 
  preProcess = c("center", "scale")
)
mylogit_lasso_acc

glmnet 

700 samples
 12 predictor
  2 classes: 'No', 'Si' 

Pre-processing: centered (12), scaled (12) 
Resampling: Cross-Validated (5 fold) 
Summary of sample sizes: 560, 560, 560, 560, 560 
Resampling results across tuning parameters:

  lambda        ROC        Sens       Spec        Accuracy   Kappa     
  0.0001000000  0.7318756  0.9142857  0.28571429  0.7257143  0.23176719
  0.0002789687  0.7318756  0.9142857  0.28571429  0.7257143  0.23176719
  0.0007782356  0.7318756  0.9142857  0.28571429  0.7257143  0.23176719
  0.0021710342  0.7318756  0.9142857  0.28571429  0.7257143  0.23176719
  0.0060565070  0.7318756  0.9142857  0.28571429  0.7257143  0.23176719
  0.0168957618  0.7327017  0.9142857  0.27619048  0.7228571  0.22125732
  0.0471338954  0.7335763  0.9326531  0.26190476  0.7314286  0.23129637
  0.1314888384  0.7347425  0.9571429  0.14285714  0.7128571  0.12505390
  0.3668127682  0.7336249  0.9918367  0.02380952  0.7014286  0.02124838
  1.0232929923  0.7298834  1.0000000  0.0

### ROC

In [15]:
set.seed(1410)
mylogit_lasso_roc <- train(
 Default~duration+amount+installment+age+
                       history.buena+history.mala+
                       purpose.auto_nuevo+purpose.auto_usado+purpose.bienes+purpose.educacion+
                       foreign.extranjero+
                       +rent.TRUE, 
  data = training, 
  method = "glmnet",
  trControl = ctrl,
  family = "binomial", 
  metric = "ROC",
  tuneGrid = expand.grid(alpha = 0,lambda=lambda_grid), 
  preProcess = c("center", "scale")
)
mylogit_lasso_roc

glmnet 

700 samples
 12 predictor
  2 classes: 'No', 'Si' 

Pre-processing: centered (12), scaled (12) 
Resampling: Cross-Validated (5 fold) 
Summary of sample sizes: 560, 560, 560, 560, 560 
Resampling results across tuning parameters:

  lambda        ROC        Sens       Spec        Accuracy   Kappa     
  0.0001000000  0.7318756  0.9142857  0.28571429  0.7257143  0.23176719
  0.0002789687  0.7318756  0.9142857  0.28571429  0.7257143  0.23176719
  0.0007782356  0.7318756  0.9142857  0.28571429  0.7257143  0.23176719
  0.0021710342  0.7318756  0.9142857  0.28571429  0.7257143  0.23176719
  0.0060565070  0.7318756  0.9142857  0.28571429  0.7257143  0.23176719
  0.0168957618  0.7327017  0.9142857  0.27619048  0.7228571  0.22125732
  0.0471338954  0.7335763  0.9326531  0.26190476  0.7314286  0.23129637
  0.1314888384  0.7347425  0.9571429  0.14285714  0.7128571  0.12505390
  0.3668127682  0.7336249  0.9918367  0.02380952  0.7014286  0.02124838
  1.0232929923  0.7298834  1.0000000  0.0

## Alternative Cutoffs

Cuando hay dos posibles categorías de resultados, otro método para aumentar la precisión de la predicción de las muestras de la clase minoritaria es determinar puntos de corte alternativos para las probabilidades predichas, lo que cambia efectivamente la definición de un evento predicho.
  
- Existen varias técnicas para determinar un nuevo punto de corte.
  
- Si hay un objetivo particular que debe cumplirse para la sensibilidad o la especificidad, este punto se puede encontrar en la curva ROC y se puede determinar el límite correspondiente.



In [16]:
evalResults <- data.frame(Default = evaluation$Default)

evalResults$Roc <- predict(mylogit_lasso_roc,
                              newdata = evaluation,
                              type = "prob")[,1]

head(evalResults)

,Default,Roc
,<fct>,<dbl>
1,Si,0.5996035
2,Si,0.7055370
3,No,0.7779962
4,No,0.7489328
5,No,0.7511926
6,Si,0.5395619


In [21]:
table(evalResults$Default)
#levels(evalResults$Default)


No Si 
70 30 

In [17]:
p_load("pROC")
rfROC <- roc(evalResults$Default, evalResults$Roc, levels = rev(levels(evalResults$Default)))
rfROC

Setting direction: controls < cases




Call:
roc.default(response = evalResults$Default, predictor = evalResults$Roc,     levels = rev(levels(evalResults$Default)))

Data: evalResults$Roc in 30 controls (evalResults$Default Si) < 70 cases (evalResults$Default No).
Area under the curve: 0.7871

In [18]:
?roc

In [23]:
rfROC <- roc(evalResults$Default, evalResults$Roc, levels = c("No","Si"))
rfROC

Setting direction: controls > cases




Call:
roc.default(response = evalResults$Default, predictor = evalResults$Roc,     levels = c("No", "Si"))

Data: evalResults$Roc in 70 controls (evalResults$Default No) > 30 cases (evalResults$Default Si).
Area under the curve: 0.7871

Otro enfoque es encontrar el punto en la curva ROC que está más cerca (es decir, la distancia más corta) al modelo perfecto (con 100\% de sensibilidad y 100\% de especificidad), que está asociado con la esquina superior izquierda de la gráfica.

In [ ]:
rfThresh <- coords(rfROC, x = "best", best.method = "closest.topleft")
rfThresh

In [ ]:
evalResults<-evalResults %>% mutate(hat_def_05=ifelse(evalResults$Roc>0.5,"Si","No"),
                                    hat_def_rfThresh=ifelse(evalResults$Roc>rfThresh$threshold,"Si","No"))

In [ ]:
with(evalResults,table(Default,hat_def_05))


In [ ]:
with(evalResults,table(Default,hat_def_rfThresh))


El límite alternativo para el modelo no se derivó de los conjuntos de entrenamiento o prueba. Es importante, especialmente para tamaños de muestra pequeños, usar un conjunto de datos independiente para derivar el límite.

 - Si se utilizan las predicciones del conjunto de entrenamiento, es probable que haya un gran sesgo optimista en las probabilidades de clase que conducirá a evaluaciones inexactas de la sensibilidad y la especificidad.

 - Si se utiliza el conjunto de prueba, ya no es una fuente imparcial para juzgar el rendimiento del modelo. Por ejemplo, Ewald (2006) descubrió mediante simulación que la derivación post hoc de los puntos de corte puede exagerar el rendimiento del conjunto de pruebas.
   
Notemos que el modelo no ha cambiado. Se están utilizando los mismos parámetros del modelo. Cambiar el límite para aumentar la sensibilidad no aumenta la efectividad predictiva general del modelo. El principal impacto que tiene un corte alternativo es hacer compensaciones entre tipos particulares de errores.

## Remuestreo

Hay dos enfoques generales:

    1. Up-sampling.  Simulates or imputes additional data points of the minority class to improve balance across classes, while 
    2. Down-sampling. Randomly reduces the number of the majority class  to improve the balance across classes.

y un híbrido: SMOTE

<div >
<img src = "sampling_methods.png" />
</div>

### Up Sampling

In [ ]:
set.seed(1103)
upSampledTrain <- upSample(x = training,
                           y = training$Default,
                           ## keep the class variable name the same:
                           yname = "Default")
dim(training)

dim(upSampledTrain)

table(upSampledTrain$Default)


In [ ]:

set.seed(1410)
mylogit_lasso_upsample <- train(Default~duration+amount+installment+age+
                       history.buena+history.mala+
                       purpose.auto_nuevo+purpose.auto_usado+purpose.bienes+purpose.educacion+
                       foreign.extranjero+
                       +rent.TRUE, 
  data = upSampledTrain, 
  method = "glmnet",
  trControl = ctrl,
  family = "binomial", 
  metric = "ROC",
  tuneGrid = expand.grid(alpha = 0,lambda=lambda_grid), 
  preProcess = c("center", "scale")
)
mylogit_lasso_upsample


### Down Sampling

In [ ]:

set.seed(1103)
downSampledTrain <- downSample(x = training,
                           y = training$Default,
                           ## keep the class variable name the same:
                           yname = "Default")
dim(training)

dim(downSampledTrain)

table(downSampledTrain$Default)


In [ ]:
set.seed(1410)
mylogit_lasso_downsample <- train(Default~duration+amount+installment+age+
                       history.buena+history.mala+
                       purpose.auto_nuevo+purpose.auto_usado+purpose.bienes+purpose.educacion+
                       foreign.extranjero+
                       rent.TRUE, 
  data = downSampledTrain, 
  method = "glmnet",
  trControl = ctrl,
  family = "binomial", 
  metric = "ROC",
  tuneGrid = expand.grid(alpha = 0,lambda=lambda_grid), 
  preProcess = c("center", "scale")
)
mylogit_lasso_downsample

### SMOTE

El synthetic minority over-sampling technique (SMOTE) (Chawla et al., 2002), utiliza tanto muestreo hacia arriba (up-sampling) como hacia abajo (down-sampling)

- Para mejorar la muestra de la clase minoritaria, SMOTE sintetiza nuevos casos. Para ello, se selecciona aleatoriamente un punto de datos de la clase minoritaria y se determinan sus K vecinos más cercanos (KNN). El nuevo punto de datos sintético es una combinación convexa aleatoria de los predictores del punto de datos seleccionado al azar y sus vecinos.

- Si bien el algoritmo SMOTE agrega nuevas muestras a la clase minoritaria a través de un muestreo ascendente, también puede reducir la muestra de casos de la clase mayoritaria a través de un muestreo aleatorio para ayudar a equilibrar el conjunto de entrenamiento.

<div >
<img src = "smote.png" />
</div>

In [ ]:
p_load("smotefamily")

predictors<-c("duration","amount","installment","age",
                       "history.buena","history.mala",
                       "purpose.auto_nuevo","purpose.auto_usado","purpose.bienes","purpose.educacion",
                       "foreign.extranjero",
                       "rent.TRUE")
head( training[predictors])


In [ ]:
smote_output = SMOTE(X = training[predictors],
                     target = training$Default)
smote_data = smote_output$data
table(training$Default)
table(smote_data$class)

In [ ]:
set.seed(1410)
mylogit_lasso_smote<- train(class~duration+amount+installment+age+
                       history.buena+history.mala+
                       purpose.auto_nuevo+purpose.auto_usado+purpose.bienes+purpose.educacion+
                       foreign.extranjero+
                       rent.TRUE,
  data = smote_data, 
  method = "glmnet",
  trControl = ctrl,
  family = "binomial", 
  metric = "ROC",
  tuneGrid = expand.grid(alpha = 0,lambda=lambda_grid), 
  preProcess = c("center", "scale")
)
mylogit_lasso_smote

## Compación

In [ ]:
testResults <- data.frame(Default = testing$Default)

testResults$logit<- predict(mylogit_caret,
                           newdata = testing,
                           type = "prob")[,1]
testResults$lasso<- predict(mylogit_lasso_roc,
                           newdata = testing,
                           type = "prob")[,1]

testResults$lasso_thresh<- predict(mylogit_lasso_roc,
                           newdata = testing,
                           type = "prob")[,1]

testResults$lasso_upsample<- predict(mylogit_lasso_upsample,
                           newdata = testing,
                           type = "prob")[,1]

testResults$mylogit_lasso_downsample<- predict(mylogit_lasso_downsample,
                           newdata = testing,
                           type = "prob")[,1]

testResults$mylogit_lasso_smote<- predict(mylogit_lasso_smote,
                           newdata = testing,
                           type = "prob")[,1]

testResults<-testResults %>% 
              mutate(logit=ifelse(logit>0.5,"Si","No"),
                     lasso=ifelse(lasso>0.5,"Si","No"),
                     lasso_thresh=ifelse(lasso_thresh>rfThresh$threshold,"Si","No"),
                     lasso_upsample=ifelse(lasso_upsample>0.5,"Si","No"),
                     mylogit_lasso_downsample=ifelse(mylogit_lasso_downsample>0.5,"Si","No"),
                     mylogit_lasso_smote=ifelse(mylogit_lasso_smote>0.5,"Si","No"),
                     )

In [ ]:
with(testResults,table(Default,logit))

In [ ]:
with(testResults,table(Default,lasso))

In [ ]:
with(testResults,table(Default,lasso_thresh))

In [ ]:
with(testResults,table(Default,lasso_upsample))

In [ ]:
with(testResults,table(Default,mylogit_lasso_downsample))

In [ ]:
with(testResults,table(Default,mylogit_lasso_smote))